In [971]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import math_ops

import matplotlib.pyplot as plt
from scipy import stats

In [972]:
route='LUX-MXP'
train_type='hist'
lambda_lasso=0
var_set='l'

if len(route)>3:
    business='airline'
else:
    business='tours'

In [973]:
def dprio_class(x):
    if x<=25:
        return 'a: x<=25'
    elif x<=50:
        return 'b: 25<x<=50'
    elif x<=75:
        return 'c: 50<x<=75'
    elif x<=100:
        return 'd: 75<x<=100'
    elif x<=125:
        return 'e: 100<x<=125'
    elif x<=150:
        return 'f: 125<x<=150'
    elif x<=175:
        return 'g: 150<x<=175'
    elif x<=200:
        return 'h: 175<x<=200'
    elif x<=225:
        return 'i: 200<x<=225'
    elif x<=250:
        return 'j: 225<x<=250'
    elif x<=275:
        return 'k: 250<x<=275'
    elif x<=300:
        return 'l: 275<x<=300'
    elif x<=325:
        return 'm: 300<x<=325'
    elif x<=350:
        return 'n: 325<x<=350'
    else:
        return 'o: 350<x'

# Neural Network

In [974]:
y_test=pd.read_csv('y_test'+'_'+route+'.csv')
y_test=y_test.rename(columns={'concat': 'id'})
out_layer=pd.read_csv('out_layer_'+route+'.csv')

out_layer['id']=y_test['id']

out_layer=out_layer.melt(id_vars='id',value_vars=[x for x in out_layer.columns if x!='id']).rename(columns={'variable': 'dprio', 'value': 'predict'})
y_test=y_test.melt(id_vars='id',value_vars=[x for x in y_test.columns if x!='id']).rename(columns={'variable': 'dprio', 'value': 'bookings'})

y_nn=y_test.merge(out_layer,on=['id','dprio'])
y_nn['abs_error']=np.abs(y_nn['bookings']-y_nn['predict'])
y_nn['dprio']=y_nn['dprio'].astype('int')
y_nn_flight=y_nn.groupby('id')['abs_error'].max().reset_index()

y_nn=y_nn.sort_values(by=['dprio','abs_error'])
y_nn.index=range(y_nn.shape[0])

y_nn=y_nn[['dprio','abs_error']]

In [975]:
X_train=pd.read_csv('X_train_'+var_set+'_'+route+'.csv')
X_train['direction:O']=X_train['direction:O'].apply(lambda x: 'O' if x==1 else 'I')
y_train=pd.read_csv('y_train'+'_'+route+'.csv')
X_test=pd.read_csv('X_test_'+var_set+'_'+route+'.csv')
X_test['direction:O']=X_test['direction:O'].apply(lambda x: 'O' if x==1 else 'I')
y_test=pd.read_csv('y_test'+'_'+route+'.csv')
y_test=y_test[[x for x in y_test.columns if x!='concat']]

y_train['id']=X_train['yearweek'].astype('int').astype('str')+'-'+X_train['weekday'].astype('str')+'-'+X_train['direction:O'].astype('str')+'-'+X_train['dep_time'].astype('str')
y_test['id']=X_test['yearweek'].astype('int').astype('str')+'-'+X_test['weekday'].astype('str')+'-'+X_test['direction:O'].astype('str')+'-'+X_test['dep_time'].astype('str')

y_train=y_train.groupby('id')[[x for x in y_train.columns if x!='id']].mean().reset_index()

y_train=y_train.melt(id_vars='id',value_vars=[x for x in y_train.columns if x!='id']).rename(columns={'variable': 'dprio', 'value': 'predict'})
y_test=y_test.melt(id_vars='id',value_vars=[x for x in y_test.columns if x!='id']).rename(columns={'variable': 'dprio', 'value': 'bookings'})

y_bench=y_test.merge(y_train,on=['id','dprio'])
y_bench['abs_error']=np.abs(y_bench['bookings']-y_bench['predict'])
y_bench['dprio']=y_bench['dprio'].astype('int')
y_bench_flight=y_bench.groupby('id')['abs_error'].max().reset_index()

#y_bench=y_bench.loc[y_bench['abs_error']>0,:]

y_bench=y_bench.sort_values(by=['dprio','abs_error'])
y_bench=y_bench[['dprio','abs_error']]

y=pd.concat([y_nn,y_bench])

y=y.sort_values(by=['dprio','abs_error']).drop_duplicates()

In [976]:
cval=0.05154512586

dprio_max=365

ks_nn=[]
ks_bench=[]

stochdom_nn=[0]*(dprio_max+1)
stochdom_bench=[0]*(dprio_max+1)

integral=[]

for dprio in range(dprio_max+1):
    
    thres_1=0
    thres_2=1
    x=y.loc[y['dprio']==dprio,'abs_error']

    x_i=y_nn.loc[y_nn['dprio']==dprio,'abs_error']
    x_diff=np.subtract.outer(x_i,x)
    x_diff=np.abs(DataFrame(x_diff))
    x_diff.index=x_i
    x_diff.columns=x
    w_nn=DataFrame(0,index=x_diff.index,columns=x_diff.columns)
    w_nn[x_diff<=thres_1]=1
    cdf_nn=w_nn.apply('sum',axis=0).reset_index()
    cdf_nn[0]=np.cumsum(cdf_nn[0])/np.sum(cdf_nn[0])
    cdf_nn=cdf_nn.loc[cdf_nn['abs_error']>=thres_2,:]

    x_i=y_bench.loc[y_bench['dprio']==dprio,'abs_error']
    x_diff=np.subtract.outer(x_i,x)
    x_diff=np.abs(DataFrame(x_diff))
    x_diff.index=x_i
    x_diff.columns=x
    w_bench=DataFrame(0,index=x_diff.index,columns=x_diff.columns)
    w_bench[x_diff<=thres_1]=1
    cdf_bench=w_bench.apply('sum',axis=0).reset_index()
    cdf_bench[0]=np.cumsum(cdf_bench[0])/np.sum(cdf_bench[0])
    cdf_bench=cdf_bench.loc[cdf_bench['abs_error']>=thres_2,:]
    
#     plt.plot(cdf_nn['abs_error'],cdf_nn[0])
#     plt.plot(cdf_bench['abs_error'],cdf_bench[0])
#     plt.show()

    ks_nn.append(np.max(cdf_nn[0]-cdf_bench[0]))
    ks_bench.append(np.max(cdf_bench[0]-cdf_nn[0]))
    
    if ks_nn[dprio]>cval:
        stochdom_nn[dprio]=1
    if ks_bench[dprio]>cval:
        stochdom_bench[dprio]=1
        
    surface=pd.concat([cdf_nn[0],cdf_bench[0]],axis=1)
    surface.columns=['nn','bench']
    integral.append(np.sum(surface.apply(lambda x: x[0]>x[1],axis=1))/surface.shape[0])

In [977]:
stochdom=DataFrame([range(dprio_max+1),stochdom_nn,stochdom_bench]).transpose()
stochdom.columns=['dprio','nn','bench']

In [978]:
[stochdom.loc[(stochdom['nn']==1)&(stochdom['bench']==0),'dprio'].unique(),
len(stochdom.loc[(stochdom['nn']==1)&(stochdom['bench']==0),'dprio'].unique())]

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47, 136, 159, 166, 167,
        170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
        183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
        209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
        222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 235,
        236, 237, 238, 239, 240, 241, 243, 244, 248, 250, 251, 252, 255,
        256, 257], dtype=int64), 132]

In [979]:
[stochdom.loc[(stochdom['nn']==0)&(stochdom['bench']==1),'dprio'].unique(),
len(stochdom.loc[(stochdom['nn']==0)&(stochdom['bench']==1),'dprio'].unique())]

[array([], dtype=int64), 0]

In [980]:
[stochdom.loc[((stochdom['nn']==1)&(stochdom['bench']==1))|((stochdom['nn']==0)&(stochdom['bench']==0)),'dprio'].unique(),
len(stochdom.loc[((stochdom['nn']==1)&(stochdom['bench']==1))|((stochdom['nn']==0)&(stochdom['bench']==0)),'dprio'].unique())]

[array([ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
         61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
         74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
         87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 168,
        169, 230, 242, 245, 246, 247, 249, 253, 254, 258, 259, 260, 261,
        262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274,
        275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287,
        288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300,
        301, 302, 303, 304, 305, 306, 307, 308, 309

In [981]:
sum([ks_nn[i]>ks_bench[i] for i in range(len(ks_nn))])/366

0.8005464480874317

In [991]:
len([x for x in integral[:360] if x>0.5 and x!=np.nan])/366

0.7185792349726776